In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install git+https://github.com/darenr/scikit-optimize
!pip install Tab2Img

  Cloning https://github.com/darenr/scikit-optimize to /tmp/pip-req-build-hwlg_it7
  Running command git clone -q https://github.com/darenr/scikit-optimize /tmp/pip-req-build-hwlg_it7
  Created wheel for scikit-optimize: filename=scikit_optimize-0.6+19.g180d6be-py2.py3-none-any.whl size=75420 sha256=afa42e8dd918fd2db2cfec83d5df817b16cda4783e231096be869111a1b056cb
  Stored in directory: /tmp/pip-ephem-wheel-cache-5lhsufdn/wheels/11/4c/bf/aa91b51947ed5367f98d5a68c59afd45a275b85e9fc4827007
Successfully built scikit-optimize


In [3]:
import pandas as pd
import time
from sklearn import metrics
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from keras import backend as K
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras import datasets, layers, models
from tensorflow.keras import applications as efn
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Reshape, GlobalAveragePooling2D, MaxPool2D, UpSampling2D,Lambda
from tensorflow.keras import optimizers
from keras.wrappers.scikit_learn import KerasClassifier
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from keras.regularizers import l2
from keras.layers.advanced_activations import PReLU
from keras.layers.core import Dense, Dropout, Activation
from sklearn.model_selection import RandomizedSearchCV
from sklearn.datasets import fetch_covtype
from tab2img.converter import Tab2Img
import time
from sklearn.metrics import confusion_matrix as conf_m
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tab2img.converter import Tab2Img
from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from tensorflow.keras import applications as efn
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Reshape, GlobalAveragePooling2D, MaxPool2D, UpSampling2D
from tensorflow.keras import optimizers
import pandas as pd
import tarfile
import scipy
import scipy.io
import matplotlib.pyplot as plt
import PIL
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import pickle
from PIL import Image
import numpy as np


data_folder_path = "/content/drive/Shareddrives/ML-final project/data/"
d1 = "abalon.csv"
d2 = "annealing.csv"
d3 = "arrhythmia.csv"
d4 = "audiology-std.csv"
d5 = "autos.csv"
d6 = "balance-scale.csv"
d7 = "baseball.csv"
d8 = "car.csv"
d9 = "cardiotocography-3clases.csv"
d10 = "cardiotocography-10clases.csv"
d11 = "conn-bench-vowel-deterding.csv"
d12 = "contrac.csv"
d13 = "dermatology.csv"
d14 = "ecoli.csv"
d15 = "energy-y1.csv"
d16 = "energy-y2.csv"
d17 = "flags.csv"
d18 = "glass.csv"
d19 = "heart-va.csv"
d20 = "iris.csv"

# Preprocessing

In [4]:
data_folder_path = "/content/drive/Shareddrives/ML-final project/data/"
data_names = [d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,d11,d12,d13,d14,d15,d16,d17,d18,d19,d20]
data_frames = []
for csv_name in data_names:
  temp_df = pd.read_csv(data_folder_path + csv_name)
  temp_df = temp_df.rename(columns = {'clase': 'class', 'symboling': 'class', 'Hall_of_Fame': 'class'}, inplace = False)
  temp_df = temp_df.fillna(temp_df.mean())
  for col_name in temp_df.columns:
    if temp_df[col_name].dtype == "object":
      temp_df[col_name] = pd.Categorical(temp_df[col_name])
      temp_df[col_name] = temp_df[col_name].cat.codes
  X = temp_df.drop('class', axis=1)
  y = temp_df['class']
  data_frames.append((X,y,len(pd.unique(temp_df['class']))))

In [5]:
def convert_to_array(X_train):
  new_x = []
  X_train_1 = np.array(X_train)
  for item in X_train_1:
    new_x.append(item)
  return  np.array(new_x)
  

def indices_to_one_hot(data, nb_classes):
    """Convert an iterable of indices to one-hot encoded labels."""
    targets = np.array(data).reshape(-1)
    return np.eye(nb_classes)[targets]


# Functions

**Metrics**

In [6]:
def sensitivity(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + K.epsilon())

    # 

def specificity(y_true, y_pred):
    true_negatives = K.sum(K.round(K.clip((1-y_true) * (1-y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1-y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())

AUC_ROC = tf.keras.metrics.AUC(
    num_thresholds=200,
    curve="ROC",
    summation_method="interpolation",
    name="ROC",
    dtype=None,
    thresholds=None,
    multi_label=False,
    num_labels=None,
    label_weights=None,
    from_logits=False,
)

AUC_PR = tf.keras.metrics.AUC(
    num_thresholds=200,
    curve="PR",
    summation_method="interpolation",
    name="PR",
    dtype=None,
    thresholds=None,
    multi_label=False,
    num_labels=None,
    label_weights=None,
    from_logits=False,
)

**Losses**

In [7]:
def weighted_loss(y_true, y_pred):
    return K.mean((weights[:,0]**(1-y_true))*(weights[:,1]**(y_true))*K.binary_crossentropy(y_true, y_pred), axis=-1)

def get_weighted_loss(weights):
    def weighted_loss(y_true, y_pred):
        return K.mean((weights[:,0]**(1-y_true))*(weights[:,1]**(y_true))*K.binary_crossentropy(y_true, y_pred), axis=-1)
    return weighted_loss


def calculating_class_weights(y_true):
    from sklearn.utils.class_weight import compute_class_weight
    number_dim = np.shape(y_true)[1]
    weights = np.empty([number_dim, 2])
    for i in range(number_dim):
        weights[i] = compute_class_weight('balanced', [0.,1.], y_true[:, i])
    return weights


def my_loss_fn_unbalnced(y_true, y_pred):
  cce = keras.losses.CategoricalCrossentropy()
  ro = 0.75
  sum = 0
  sum = tf.cast(sum, dtype='float32')
  r = tf.TensorArray(y_pred.dtype, 0, dynamic_size=True)
  for item in y_pred:
    result = tf.math.top_k(item, KK)
    value = result[0][0]
    i = 1
    mean = 0
    while i < KK:
       mean += result[0][i]
       i += 1
    mean = value - mean * (1 / (KK - 1))
    mean = tf.cast(mean, dtype='float32')
    sum += mean

  return ro*cce(y_true, y_pred) + (((1-ro)*sum)  / BATCH_SIZE )

In [8]:
def print_plots(history):
  #  "Accuracy"
  plt.plot(history.history['accuracy'])
  plt.plot(history.history['val_accuracy'])
  plt.title('model accuracy')
  plt.ylabel('accuracy')
  plt.xlabel('epoch')
  plt.legend(['train', 'validation'], loc='upper left')
  plt.show()
  # "Loss"
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('model loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train', 'validation'], loc='upper left')
  plt.show()

def eval_metrics(y_test, predictions):
  confusion_matrix = conf_m(y_test.argmax(axis=1), predictions.argmax(axis=1))
  # scores = distiller.evaluate([X_test,X_test], y_test, batch_size=15)

  FP = confusion_matrix.sum(axis=0) - np.diag(confusion_matrix)  
  FN = confusion_matrix.sum(axis=1) - np.diag(confusion_matrix)
  TP = np.diag(confusion_matrix)
  TN = confusion_matrix.sum() - (FP + FN + TP)

  # Sensitivity, hit rate, recall, or true positive rate
  TPR = np.nan_to_num(TP/(TP+FN))
  # Specificity or true negative rate
  TNR = np.nan_to_num(TN/(TN+FP))
  # Precision or positive predictive value
  PPV = np.nan_to_num(TP/(TP+FP))

  # Negative predictive value
  NPV = TN/(TN+FN)
  
  # Fall out or false positive rate
  FPR = FP/(FP+TN)
  # False negative rate
  FNR = FN/(TP+FN)
  # False discovery rate
  FDR = FP/(TP+FP)

  # Overall accuracy
  ACC = np.nan_to_num((TP+TN)/(TP+FP+FN+TN))

  AUC_ROC_m = AUC_ROC(y_test, predictions)
  AUC_PR_m = AUC_PR(y_test, predictions)

  return np.average(ACC), np.average(TPR), np.average(TNR), np.average(PPV), float(AUC_ROC_m), float(AUC_PR_m)

# Model by the article - A More Tolerant Teacher Educates Better Students

Models

In [9]:

def build_keras_base(hidden_layers = [500, 256, 64], dropout_rate = 0, 
                     l2_penalty = 0.1, optimizer = 'adam',
                     n_class = 18, verbose = 1):

    model=Sequential()
    for layers in hidden_layers:
      model.add(Dense(layers, activation='relu', kernel_regularizer = l2(l2_penalty)))  
      model.add(Dropout(dropout_rate))
      model.add(BatchNormalization())

    model.add(Dense(n_class, activation='softmax'))
    
    if verbose == 1:
      loss = my_loss_fn_unbalnced
    else:
      loss = 'categorical_crossentropy'
    
    model.compile(loss = loss, optimizer = optimizer, metrics = ['accuracy'])   
    return model


class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def call(self, x):
      return self.student(x)

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        (x_1,x_2), y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x_1, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x_2, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)
            distillation_loss = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions / self.temperature, axis=1),
            )
            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        (x_1,x_2), y = data

        # Compute predictions
        y_prediction = self.student(x_2, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results



In [ ]:


num_folds_outter = 10
BATCH_SIZE = 32

d = { 'Params':[], 'Accuracy':[],	'TPR':[],	'TNR':[],	'Precision':[],	'AUC':[], 'Curve':[], 'Time train':[], 'Time test':[]}
df_marks = pd.DataFrame(d)

kfold_outter = KFold(n_splits=num_folds_outter, shuffle=True)

for index, df in enumerate(data_frames):
  # if index == 19:
    print('============',index,'============')
    for train_valid, test in kfold_outter.split(df[0], df[1]):

      num_of_class = df[2]

      X_train = df[0].iloc[train_valid]
      y_train = df[1].iloc[train_valid]
      X_test = df[0].iloc[test]
      y_test = df[1].iloc[test]

      X_train = convert_to_array(X_train)
      y_train = np.array(y_train)
      y_train = y_train.reshape(-1,1)
      y_train = indices_to_one_hot(y_train, num_of_class)

      X_test = convert_to_array(X_test)
      y_test = np.array(y_test)
      y_test = y_test.reshape(-1,1)
      y_test = indices_to_one_hot(y_test, num_of_class)

      model_keras = KerasClassifier(
      build_fn = build_keras_base,
      n_class = num_of_class,
      verbose = 0
  )
      class_weights = calculating_class_weights(y_train)
      earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)

      callbacks = [earlystop]

      dropout_rate_opts  = [0, 0.2, 0.5]
      hidden_layers_opts = [[500, 256], [1000, 500, 256], [100, 500]]
      l2_penalty_opts = [0, 0.01, 0.1, 0.5]
      keras_param_options = {
          'hidden_layers': hidden_layers_opts,
          'dropout_rate': dropout_rate_opts,  
          'l2_penalty': l2_penalty_opts
      }

      rs_keras = RandomizedSearchCV( 
        model_keras, 
        param_distributions = keras_param_options,
        scoring = 'neg_log_loss',
        n_iter = 3, 
        cv = 3,
        n_jobs = -1,
        verbose = 1
    )
      
      keras_fit_params = {   
        'callbacks': callbacks,
        'epochs': 20,
        'batch_size': 32,
        'validation_split':0.2,
        'verbose': 0
    }
      start = time.time()

      num_folds_outter = 10

      if num_of_class < 5:
        KK = 2
      elif num_of_class < 10:
        KK = 3
      else:
        KK = 4

      rs_keras.fit(X_train, y_train,  **keras_fit_params)

      teacher = rs_keras.best_estimator_.model

      opt_params = []
      for param, value in rs_keras.best_params_.items():
          opt_params.append(value)
          # print('\t{}: {}'.format(param, value))

      generations = 4

      for i in range(4):

        student = build_keras_base(hidden_layers = opt_params[1], dropout_rate = opt_params[2], 
                        l2_penalty = opt_params[0], optimizer = 'adam',
                        n_class = num_of_class, verbose = 0)
        

        earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
        # Initialize and compile distiller

        distiller = Distiller(student=student, teacher=teacher)
        distiller.compile(
            optimizer=keras.optimizers.Adam(),
            metrics=['accuracy'],
            student_loss_fn=keras.losses.CategoricalCrossentropy(), # get_weighted_loss(class_weights)
            distillation_loss_fn=keras.losses.KLDivergence(),
            alpha=0.1,
            temperature=10,
        )

        # Distill teacher to student
        distiller.fit([X_train,X_train], y_train, epochs=5, verbose=0)

        teacher = distiller

      # Evaluate student on test dataset
      end = time.time()
      training_time = (end - start)
      start = time.time()
      predictions = distiller.predict(X_test)
      end = time.time()
      confusion_matrix = conf_m(y_test.argmax(axis=1), predictions.argmax(axis=1))
      # scores = distiller.evaluate([X_test,X_test], y_test, batch_size=15)

      FP = confusion_matrix.sum(axis=0) - np.diag(confusion_matrix)  
      FN = confusion_matrix.sum(axis=1) - np.diag(confusion_matrix)
      TP = np.diag(confusion_matrix)
      TN = confusion_matrix.sum() - (FP + FN + TP)

      # Sensitivity, hit rate, recall, or true positive rate
      TPR = TP/(TP+FN)
      # Specificity or true negative rate
      TNR = TN/(TN+FP) 
      # Precision or positive predictive value
      PPV = np.nan_to_num(TP/(TP+FP))

      # Negative predictive value
      NPV = TN/(TN+FN)
      
      # Fall out or false positive rate
      FPR = FP/(FP+TN)
      # False negative rate
      FNR = FN/(TP+FN)
      # False discovery rate
      FDR = FP/(TP+FP)

      # Overall accuracy
      ACC = (TP+TN)/(TP+FP+FN+TN)

      AUC_ROC_m = AUC_ROC(y_test, predictions)
      AUC_PR_m = AUC_PR(y_test, predictions)

      new_row = {'Params':opt_params, 'Accuracy':np.average(ACC),	'TPR':np.average(TPR),	'TNR':np.average(TNR),	'Precision':np.average(PPV),	'AUC':float(AUC_ROC_m),
        'Curve':float(AUC_PR_m), 'Time train':training_time, 'Time test':end -start}



      df_marks =  df_marks.append(new_row, ignore_index=True)

    df_marks.to_csv(data_folder_path+'results_2.csv', sep='\t', encoding='utf-8')


      



df_marks.to_csv(data_folder_path+'results_2.csv', sep='\t', encoding='utf-8')
  
df_marks

# Improvment

**A Batter Teacher**

In [13]:

def get_model(UP, SIZE, num_classes=2):


  base_model = efn.ResNet50(weights='imagenet', include_top=False, input_shape=(SIZE,SIZE,3))

     

  for layer in base_model.layers:
      if isinstance(layer, BatchNormalization):
          layer.trainable = True
      else:
          layer.trainable = False

  model=Sequential()
  model.add(UpSampling2D(size=(UP, UP)))


  model.add(base_model)
  model.add(GlobalAveragePooling2D())
  model.add(Dense(500, activation='relu'))
  model.add(Dropout(.4))
  model.add(BatchNormalization())

  model.add(BatchNormalization())
  model.add(Dense(num_classes, activation='softmax'))

  model.compile(loss=my_loss_fn_unbalnced, optimizer='adam', metrics=['accuracy'])
 

  return model

**Experiments**

In [14]:

num_folds_outter = 10
d = {'GEN': [],'VAL':[], 'Accuracy':[],	'TPR':[],	'TNR':[],	'Precision':[],	'AUC':[], 'Curve':[], 'Time train':[], 'Time test':[]}
df_marks = pd.DataFrame(d)
BATCH_SIZE = 64

kfold_outter = KFold(n_splits=num_folds_outter, shuffle=True)
for i in range(20):
  if i == 19:
    df = data_frames[i]

    counter = 1
    for train_valid, test in kfold_outter.split(df[0], df[1]):

        
          num_of_class = df[2]

          n_class = num_of_class

          train = df[0].values
          target = df[1].values

          model = Tab2Img()
          images = model.fit_transform(train, target)

          X_train_2 = train[train_valid]
          X_test_2 = train[test]

          X_train = images[train_valid]
          y_train = df[1].iloc[train_valid]
          X_test = images[test]
          y_test = df[1].iloc[test]

          X_train = convert_to_array(X_train)
          y_train = np.array(y_train)
          y_train = y_train.reshape(-1,1)
          y_train = indices_to_one_hot(y_train, num_of_class)

          X_test = convert_to_array(X_test)
          y_test = np.array(y_test)
          y_test = y_test.reshape(-1,1)
          y_test = indices_to_one_hot(y_test, num_of_class)

          X_train = np.repeat(X_train[..., np.newaxis], 3, -1)
          X_test = np.repeat(X_test[..., np.newaxis], 3, -1)


          if num_of_class < 5:
            KK = 2
          elif num_of_class < 10:
            KK = 3
          else:
            KK = 4

          print('=======================================')
          if X_train.shape[1] == 6:
            model = get_model(6,36,num_of_class)
          elif X_train.shape[1] == 17:
            model = get_model(2,34,num_of_class)
          elif X_train.shape[1] == 8:
            model = get_model(4,32,num_of_class)
          elif X_train.shape[1] == 2:
            model = get_model(16,32,num_of_class)
          elif X_train.shape[1] == 4:
            model = get_model(8,32,num_of_class)
          elif X_train.shape[1] == 3:
            model = get_model(11,33,num_of_class)
          elif X_train.shape[1] == 5:
            model = get_model(7,35,num_of_class)
          
          if  X_train.shape[0] > 800:
            epoches_fit = 25
            batch_size_f = 64
          else:
            epoches_fit = 50
            batch_size_f = 32



          start_1 = time.time()
          history = model.fit(
                              X_train,y_train,
                              batch_size = 64,
                              validation_split=0.2,
                              epochs = 1,
                              shuffle=True,
                              verbose = 1

                              )
          end_1 = time.time()
          train_time = end_1 - start_1

          start = time.time()
          predictions = model.predict(X_test)
          end = time.time()

          ACC, TPR, TNR, PPV, AUC_ROC_m, AUC_PR_m = eval_metrics(y_test, predictions)
          
          new_row = {'GEN': 0,'VAL':counter, 'Accuracy':ACC,	'TPR':TPR,	'TNR':TNR,	'Precision':PPV,	'AUC':AUC_ROC_m,
            'Curve':AUC_PR_m, 'Time train':train_time, 'Time test':(end-start)}

          # print(new_row)

          generations = 2

          teacher = model

          for i in range(5):
            # start = time.time()
            student = build_keras_base(n_class = num_of_class,verbose = 0)


            earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
            # Initialize and compile distiller
            # class_weights = calculating_class_weights(y_train)

            distiller = Distiller(student=student, teacher=teacher)
            distiller.compile(
                optimizer=keras.optimizers.Adam(),
                metrics=['accuracy',"Precision",AUC_ROC,AUC_PR,sensitivity, specificity],
                student_loss_fn=keras.losses.CategoricalCrossentropy(from_logits=False),
                distillation_loss_fn=keras.losses.KLDivergence(),
                alpha=0.1,
                temperature=10,
            )

            # Distill teacher to student
            start_1 = time.time()
            distiller.fit([X_train,X_train_2], y_train, epochs=epoches_fit, verbose=0, batch_size = batch_size_f)
            end_1 = time.time()

            train_time += (end_1 - start_1)

            start = time.time()
            predictions = distiller.predict(X_test_2)
            end = time.time()

            ACC, TPR, TNR, PPV, AUC_ROC_m, AUC_PR_m = eval_metrics(y_test, predictions)

            if new_row['Curve'] < AUC_PR_m:

              new_row = {'GEN': i+1,'VAL':counter, 'Accuracy':ACC,	'TPR':TPR,	'TNR':TNR, 'Precision':PPV,	'AUC':AUC_ROC_m,
                'Curve':AUC_PR_m, 'Time train':train_time, 'Time test':end-start}

            teacher = distiller
            X_test = X_test_2 
            X_train = X_train_2 
            
          counter += 1
            
          df_marks =  df_marks.append(new_row, ignore_index=True)

    df_marks.to_csv(data_folder_path+'results_3.csv', sep='\t', encoding='utf-8')
      
      

df_marks.head()

2/2 [==============================] - 20s 1s/step - loss: 0.9678 - accuracy: 0.5093 - val_loss: 1.0789 - val_accuracy: 0.0000e+00


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in true_divide


+++++++++++++++++++++++++++++++++
+++++++++++++++++++++++++++++++++
2/2 [==============================] - 6s 1s/step - loss: 1.0234 - accuracy: 0.4537 - val_loss: 1.7488 - val_accuracy: 0.0000e+00
+++++++++++++++++++++++++++++++++
+++++++++++++++++++++++++++++++++
2/2 [==============================] - 6s 987ms/step - loss: 0.9451 - accuracy: 0.4815 - val_loss: 0.2536 - val_accuracy: 0.9630
+++++++++++++++++++++++++++++++++
+++++++++++++++++++++++++++++++++
2/2 [==============================] - 6s 1s/step - loss: 1.1669 - accuracy: 0.3333 - val_loss: 0.7300 - val_accuracy: 0.5926
+++++++++++++++++++++++++++++++++
+++++++++++++++++++++++++++++++++
2/2 [==============================] - 6s 963ms/step - loss: 1.0368 - accuracy: 0.4537 - val_loss: 1.3650 - val_accuracy: 0.0000e+00
+++++++++++++++++++++++++++++++++
+++++++++++++++++++++++++++++++++
2/2 [==============================] - 6s 1s/step - loss: 1.1470 - accuracy: 0.4537 - val_loss: 0.1968 - val_accuracy: 1.0000
++++++++++++++++

,GEN,VAL,Accuracy,TPR,TNR,Precision,AUC,Curve,Time train,Time test
0,4.0,1.0,1.000000,1.000000,1.000000,1.000000,0.999833,0.999667,37.728694,0.101182
1,4.0,2.0,0.911111,0.916667,0.939394,0.888889,0.998845,0.997730,22.103323,0.099791
2,4.0,3.0,0.955556,0.916667,0.962963,0.952381,0.999956,0.999912,23.371987,0.103387
3,2.0,4.0,1.000000,1.000000,1.000000,1.000000,0.999622,0.999259,17.113719,0.100307
4,4.0,5.0,1.000000,1.000000,1.000000,1.000000,0.999378,0.998807,22.642606,0.095472


In [15]:

df_marks.head()

,GEN,VAL,Accuracy,TPR,TNR,Precision,AUC,Curve,Time train,Time test
0,4.0,1.0,1.000000,1.000000,1.000000,1.000000,0.999833,0.999667,37.728694,0.101182
1,4.0,2.0,0.911111,0.916667,0.939394,0.888889,0.998845,0.997730,22.103323,0.099791
2,4.0,3.0,0.955556,0.916667,0.962963,0.952381,0.999956,0.999912,23.371987,0.103387
3,2.0,4.0,1.000000,1.000000,1.000000,1.000000,0.999622,0.999259,17.113719,0.100307
4,4.0,5.0,1.000000,1.000000,1.000000,1.000000,0.999378,0.998807,22.642606,0.095472


In [49]:
import numpy as np
import pandas as pd

df_time = pd.DataFrame({'time': []})

features = [ 3,31, 262, 159, 25, 4, 16, 6, 21, 21, 11, 9, 34, 7, 8, 8, 28, 9, 12, 4]

for feature in features:

  for i in range(10):
    df = pd.DataFrame(np.random.uniform(0,1,size=(1000, 31)))
    df_y = pd.DataFrame(np.random.randint(0,3,size=(1000, 1)))

    X_train = convert_to_array(df)

    model = Tab2Img()
    X_train = model.fit_transform(X_train, df_y.values)

    model = build_keras_base()

    # if X_train.shape[1] == 6:
    #   model = get_model(6,36,num_of_class)
    # elif X_train.shape[1] == 17:
    #   model = get_model(2,34,num_of_class)
    # elif X_train.shape[1] == 8:
    #   model = get_model(4,32,num_of_class)
    # elif X_train.shape[1] == 2:
    #   model = get_model(16,32,num_of_class)
    # elif X_train.shape[1] == 4:
    #   model = get_model(8,32,num_of_class)
    # elif X_train.shape[1] == 3:
    #   model = get_model(11,33,num_of_class)
    # elif X_train.shape[1] == 5:
    #   model = get_model(7,35,num_of_class)

    X_train = np.repeat(X_train[..., np.newaxis], 3, -1)

    start = time.time()
    predict = model.predict(X_train)
    end = time.time()
    row = {'time': end-start}
    df_time =  df_time.append(row, ignore_index=True)

df_time.to_csv(data_folder_path+'results_4.csv', sep='\t', encoding='utf-8')

In [45]:
df_time


,time
0,1.933589
1,1.950438
2,2.003186
3,2.041372
4,2.076138
5,2.032463
6,2.542524
7,2.060372
8,2.041564
9,1.945109


# Statisticl tests - friedman's

In [ ]:
#statisticl tests - friedman's

# !pip install scikit_posthocs
import pandas as pd
from scipy.stats import friedmanchisquare
import scikit_posthocs as sp

avg_scores_base = [0.575355089,0.605236263,0.617489813,0.610611191,0.608549949,0.621841123,0.656721626,0.70341512,0.741710655,0.754682287
,0.760062099,0.745912285,0.733846726,0.736557243,0.741724752,0.748451151,0.749622821,0.747659547,0.743561954,0.741126907]

avg_scores_teacher = [0.7598,0.7968,0.3492,0.6384,0.2861,0.958,0.8447,0.9744,0.9624,0.8672,0.8385,0.5288,0.9923,0.7943,0.9318,0.9355,0.456,0.609,0.3514,0.9186]

avg_scores_improvement = [0.7577053,0.9572435,0.7112947,0.8842785,0.5749621,0.853707,0.983879,0.9998219,0.9818908,0.9268775,0.9981706,0.5762453,0.9932229,0.9168473,0.9534546
,0.9013419,0.5699038,0.7665592,0.3949459,0.9974408]

avg = []
for i in range(20):
  avg.append((avg_scores_base[i],avg_scores_teacher[i],avg_scores_improvement[i]))
stat, p = friedmanchisquare(*avg)
print('Statistics=%.10f, p=%.10f' % (stat, p))

names = ["base","teacher", "improved"]

hoc_test=sp.posthoc_nemenyi_friedman(avg)
col_name={}
for name in names:
    col_name.update({names.index(name):name})
hoc_test.rename(columns=col_name,index=col_name,inplace = True)
print(hoc_test)



Statistics=30.6380952381, p=0.0442186293
             base  teacher  improved
base      1.00000  0.51037   0.00100
teacher   0.51037  1.00000   0.03067
improved  0.00100  0.03067   1.00000
